In [63]:
import numpy as np
from scipy.linalg import expm
from scipy.optimize import minimize

seniority 2 specific decomp.

### The Adjoint Representation

A Lie algebra is a set of operators (our basis $\{A_p\}$) that is closed under a binary operation called the Lie bracket, which for our purposes is the commutator $[A_i, A_j] = A_iA_j - A_jA_i$.

The "closure" property means that the commutator of any two operators in the basis can be written as a linear combination of other operators within that same basis. We can express this formally with **structure constants**, denoted by $\gamma_{k}^{ij}$:

$$
[A_i, A_j] = \sum_{k=1}^{N} \gamma_{k}^{ij} A_k
$$

Here, $N$ is the dimension of the algebra (in our case, $N=5$). The structure constants uniquely define the algebra.

The **adjoint representation** is a way to map each abstract operator $A_i$ to a concrete $N \times N$ matrix, which we'll call $M_i$. This mapping is defined directly by the structure constants. The element in the $k$-th row and $j$-th column of the matrix $M_i$ is given by:

$$
(M_{i})_{k,j} = \gamma_{k}^{ij}
$$

This definition has a very practical interpretation: The $j$-th column of the matrix $M_i$ is simply the list of coefficients of the basis operators $\{A_k\}$ in the expansion of the commutator $[A_i, A_j]$.

To generate these matrices programmatically, we first need to encode the commutation rules of our algebra. We can represent the basis operators $A_1, \dots, A_5$ and store their commutation relations in a dictionary. The result of each commutator is a list of coefficients corresponding to the basis $(A_1, A_2, A_3, A_4, A_5)$.

For example, the rule $[A_1, A_2] = A_3$ is encoded as the coefficient vector `[0, 0, 1, 0, 0]`. The rule $[A_1, A_4] = -A_3$ is encoded as `[0, 0, -1, 0, 0]`.


In [ ]:
def generate_adjoint_matrices():
    """
    Generates the 5x5 adjoint matrices for the seniority-2 Lie algebra
    by programmatically applying the definition based on structure constants.
    """
    num_operators = 5
    
    # Define the basis operators by their index (0 to 4 for A1 to A5)
    basis_indices = range(num_operators)

    # Store the known commutation relations as a dictionary of dictionaries.
    # commutators[i][j] gives the result of [A_{i+1}, A_{j+1}]
    # The result is a vector of coefficients for the basis (A1, ..., A5).
    commutators = {i: {} for i in basis_indices}

    # Helper function to set commutators, automatically handling antisymmetry
    def set_commutator(i, j, result_vector):
        # [A_i, A_j] = result
        commutators[i][j] = np.array(result_vector)
        # [A_j, A_i] = -result
        commutators[j][i] = -np.array(result_vector)

    # --- Define the algebra based on the known relations ---
    # [A1, A2] = A3
    set_commutator(0, 1, [0, 0, 1, 0, 0])
    # [A1, A3] = A4
    set_commutator(0, 2, [0, 0, 0, 1, 0])
    # [A1, A4] = -A3
    set_commutator(0, 3, [0, 0, -1, 0, 0])
    # [A2, A3] = -A5
    set_commutator(1, 2, [0, 0, 0, 0, -1])
    # [A2, A5] = A3
    set_commutator(1, 4, [0, 0, 1, 0, 0])
    # [A3, A4] = -A5
    set_commutator(2, 3, [0, 0, 0, 0, -1])
    # [A3, A5] = A4
    set_commutator(2, 4, [0, 0, 0, 1, 0])
    # [A4, A5] = A3
    set_commutator(3, 4, [0, 0, 1, 0, 0])

    # --- Generate the Adjoint Matrices ---
    adjoint_matrices = []
    for i in basis_indices:  # For each matrix M_i we want to build
        M_i = np.zeros((num_operators, num_operators))
        for j in basis_indices:  # For each column j of M_i
            # The j-th column of M_i is the result of [A_i, A_j]
            # Check if the commutator is defined, otherwise it's zero
            if j in commutators[i]:
                result_vector = commutators[i][j]
                M_i[:, j] = result_vector
        adjoint_matrices.append(M_i)
        
    return adjoint_matrices

In [ ]:
adjoint_matrices = generate_adjoint_matrices()

print("--- Adjoint Matrices ---\n")
for i, M in enumerate(adjoint_matrices):
    print(f"--- M_{i+1} ---")
    print(M)
    print("-" * 15 + "\n")

--- Adjoint Matrices ---

--- M_1 ---
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  1.  0. -1.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
---------------

--- M_2 ---
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.]]
---------------

--- M_3 ---
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  1.]
 [ 0.  1.  0. -1.  0.]]
---------------

--- M_4 ---
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]]
---------------

--- M_5 ---
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0. -1.  0. -1.  0.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
---------------



In [ ]:
def solve_decomposition(theta_coeffs:list):
    """
    Solves the Lie algebra decomposition problem for a given theta.

    e^{theta * (A1 + A2)} = product_p e^{t_p(theta) * A_p}

    This function uses the adjoint matrix representation to find the
    parameters t_p(theta).

    Args:
        theta (float): The parameter for the operator exponential.

    Returns:
        scipy.optimize.OptimizeResult: The result from the optimizer,
                                       containing the solution for t_p.
    """

    # 1. Define the Lie Algebra Adjoint Matrices (5x5)
    # The basis is ordered as (A1, A2, A3, A4, A5)
    # The element (M_i)_kj is the structure constant gamma^ij_k
    # from the commutator [A_i, A_j] = sum_k gamma^ij_k * A_k

    # 2. Compute the Left-Hand Side (The Target Matrix)
    M_sum = np.zeros((5, 5))
    for i in range(5):
        M_sum += theta_coeffs[i] * adjoint_matrices[i]
    U_target = expm(M_sum)
    
    # 3. Define the Cost Function for Numerical Optimization
    def cost_function(t_params):
        """
        Calculates the difference between the target matrix and the product.

        Args:
            t_params (list or np.array): The list of five parameters [t1, t2, t3, t4, t5].

        Returns:
            float: The squared Frobenius norm of the difference, which the
                   optimizer will try to minimize.
        """
        # Calculate the product on the right-hand side:
        # U_prod = exp(t1*M1) @ exp(t2*M2) @ ... @ exp(t5*M5)
        # Note: The '@' operator is used for matrix multiplication.
        U_prod = np.identity(5) # Start with identity matrix
        for i in range(5):
            U_prod = U_prod @ expm(t_params[i] * adjoint_matrices[i])

        # Calculate the error (cost)
        error_matrix = U_prod - U_target
        cost = np.linalg.norm(error_matrix, 'fro')**2
        return cost

    # 4. Perform the Numerical Optimization
    # We need an initial guess for the parameters t_p.
    # A simple guess is to start with all zeros.
    initial_guess = np.zeros(5)

    # Use the L-BFGS-B optimizer to find the t_p that minimize the cost function.
    result = minimize(cost_function, initial_guess, method='L-BFGS-B')

    return result, U_target, adjoint_matrices

In [54]:
THETA_COEFFS = [0.200, 0.300, 0.100, -0.100, 0.050]
# THETA_COEFFS = [0.5, 0.5, 0, 0, 0]
print(f"--- Solving for theta_coeffs = {THETA_COEFFS} ---\n")

# Solve the decomposition
optimization_result, U_target, matrices = solve_decomposition(THETA_COEFFS)

# Extract the solution
t_solution = optimization_result.x

# Print the results
if optimization_result.success:
    print("Optimization successful!")
    print(f"Found solution for t_p:")
    for i in range(5):
        print(f"  t_{i+1} = {t_solution[i]:.8f}")
    print(f"\nFinal cost (error): {optimization_result.fun:.2e}\n")

else:
    print("Optimization failed.")
    print("Message:", optimization_result.message)

# 5. Final Verification
print("--- Verification ---")
print("Comparing the target matrix with the matrix from the solution.\n")

# Calculate the product matrix using the found solution
U_solution = np.identity(5)
for i in range(5):
    U_solution = U_solution @ expm(t_solution[i] * matrices[i])

print("Target Matrix (LHS):")
print(np.round(U_target, 5))
print("\nProduct Matrix from Solution (RHS):")
print(np.round(U_solution, 5))

# Calculate the final difference
final_difference = np.linalg.norm(U_solution - U_target, 'fro')
print(f"\nFrobenius norm of the difference: {final_difference:.2e}")

--- Solving for theta_coeffs = [0.2, 0.3, 0.1, -0.1, 0.05] ---

Optimization successful!
Found solution for t_p:
  t_1 = 0.18841075
  t_2 = 0.30579877
  t_3 = 0.06333541
  t_4 = -0.09785498
  t_5 = 0.04797337

Final cost (error): 1.87e-03

--- Verification ---
Comparing the target matrix with the matrix from the solution.

Target Matrix (LHS):
[[ 1.       0.       0.       0.       0.     ]
 [ 0.       1.       0.       0.       0.     ]
 [-0.37963  0.15661  0.94302 -0.25469  0.18347]
 [-0.12628  0.01558  0.12709  0.97766  0.11276]
 [ 0.08345  0.06848 -0.39909 -0.04844  0.95663]]

Product Matrix from Solution (RHS):
[[ 1.       0.       0.       0.       0.     ]
 [ 0.       1.       0.       0.       0.     ]
 [-0.37373  0.16156  0.92438 -0.24971  0.19283]
 [-0.13539  0.0163   0.12117  0.96983  0.10151]
 [ 0.08237  0.07928 -0.39333 -0.04143  0.98576]]

Frobenius norm of the difference: 4.33e-02
